##### Imports

In [ ]:
import os
# TENSORFLOW LOGS:
# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [ ]:
import os
import math
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.python.keras.api import keras

from IPython.display import clear_output

from hpcscripts.sharedutils.trainingutils import LoadModel, SetLowTFVerbose, MakeSinglePrediction, CreateWindowGenerator, GetFileList, TrainModel
from hpcscripts.sharedutils.nomalization import DF_Nomalize, denorm
from hpcscripts.sharedutils.modelutils import SelectModelPrompt
from hpcscripts.trainers.anntrainer import CreateANNModel, ImportCombinedTrainingData
from hpcscripts.trainers.modeldefinitions import  MODEL_DEFINITIONS
from hpcscripts.trainers import modeldefinitions as mdef
from hpcscripts.trainers import anntrainer
from hpcscripts.option import pathhandler as ph
from hpcscripts.option import globalparams as G_PARAMS

# Codes

#### Functions Defninition

In [ ]:
def create_binary(digits_num: int, init_val: int=0):
    return [init_val for i in range (digits_num)]

def increment_binary(binary: list):
    hold = 0
    for i in range (len(binary)):
        index = len(binary) - i - 1
        
        if index == 0 and binary[index] == 1 and hold > 0:
            return -1
        if i == 0:
            binary [index] += 1

        binary [index] += hold
        hold = 0

        if binary[index] > 1:
            binary[index] = 0
            hold = 1
    
    return binary

def create_possibility_mask(digits_num: int):
    possibilities = []
    binary = create_binary(digits_num)

    while True:

        binary = increment_binary(binary)

        if binary == -1:
            break

        possibilities.append(binary.copy())

    return possibilities

def create_possible_features(feature_list: list):
    digits_num = len (feature_list)
    possi_mask = create_possibility_mask(digits_num)

    feature_possibs = []
    for possi in possi_mask:
        new_features = []
        for mask, feature in zip(possi, feature_list):
            if mask == 1:
                new_features.append(feature)

        feature_possibs.append(new_features)
    
    return feature_possibs, possi_mask

In [ ]:
def print_list(the_list: list):
    for element in the_list:
        print (element)
        
def bin_to_index(binary: str = "1010"):
    index = -1
    for n, letter in enumerate(binary[::-1]):
        if letter == "1":
            index += 2**n
    return index

#### Create Possible Labels Combinations

In [ ]:
feature_list = [
                'hralt_m', 'theta_rad', 'aoac_rad', 'cas_mps', 'hdot_1_mps',
                'gamma_error_rad', 'g_err_d_rad', 'g_err_i_rad',
                'tailwind_mps', 'crosswind_mps',
                'use_flap'
                ]
                # 'flap_4_bool', 'flap_5_bool', 'flap_6_bool'
# label_list = ["hralt_m", "theta_rad", "aoac_rad", "cas_mps"]
# feature_list = ["hralt_m", "theta_rad"]

feature_possibs, possi_mask = create_possible_features(feature_list)

for i, feature_poss in enumerate (feature_possibs):
    if 'use_flap' in feature_poss:
        feature_poss.remove('use_flap')
        feature_poss = feature_poss + ['flap_4_bool', 'flap_5_bool', 'flap_6_bool']

        feature_possibs[i] = feature_poss

print ("Num. of possibilities: {}".format(len(feature_possibs)))
print_list(feature_possibs)

In [ ]:
# feature_possibs[bin_to_index("1011")]

### Search for the best features

#### Functions Definition

In [ ]:
def search_features(model_id: str, itter_times:int, collected_list:list):
    for features in feature_possibs:
        val_data = {}
        val_data_avg = {}

        feature_summary = {}

        G_PARAMS.FEATURE_COLUMNS = features
        for i in range (itter_times):
            clear_output(wait=True)
            print ("i = {}, feature -> {}".format(i, features))
            model, history = anntrainer.run(
                model_id,
                save_model = False,
                return_model = True
            )

            print ()

            for key in history.history.keys():
                if not key.startswith('val'):
                    continue
                min_value = min (history.history[key])
                min_epoch = history.history[key].index(min_value)
                # print ("Minimum of {:<26} ->  {} \t on epoch ->  {}".format(key, min_value, min_index)) 

                val_data[key] = val_data.get(key, [])
                val_data[key].append(min_value)

                # Min Index
                val_data[key + '_epoch'] = val_data.get(key + '_epoch', [])
                val_data[key + '_epoch'].append(min_epoch)
                

        for key in val_data.keys():
            val_data_avg[key] = np.average (val_data[key])

        feature_summary['features']     = features
        feature_summary['performance']  = val_data_avg

        collected_list.append(feature_summary)
        
        # Save collected list
        with open("features_search.pkl", 'wb') as f:
            pickle.dump(collected_list, f)


    # Save collected list
    with open("features_search.pkl", 'wb') as f:
        pickle.dump(collected_list, f)
    print ("picke saved")


#### Codes

In [ ]:
model_id = 'simp_dense'
itter_times = 5
collected_list = []

if False:
    with open("features_search.pkl", 'rb') as f:
        collected_list = pickle.load(f)    

if True: 
    search_features(model_id, itter_times, collected_list)

In [ ]:
with open("features_search.pkl", 'rb') as f:
    loaded_result = pickle.load(f)

loaded_result

In [ ]:
if False:
    print ("Loaded from pkl: {}".format(len (loaded_result)))

    sorted_features = sorted(loaded_result, key=lambda x: x['performance']['val_mean_absolute_error'])
    best_10s = sorted_features[:10]

In [ ]:
# feat_counts = {}

# for data in best_10s:
#     for feature in data['features']:
#         feat_counts[feature] = feat_counts.get(feature, 0) + 1

# feat_counts

In [ ]:
# best_10s